In [1]:
import datetime
import tarfile

import boto3
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import sagemaker
!pip install xlrd
!pip install pmdarima


sm_boto3 = boto3.client('sagemaker')

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

print('Using bucket ' + bucket)
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
connection = pg.connect(user = 'postgres',
password = 'sa2dbroot',
host = 'sa2-db.cvl1on5n4pwi.us-east-1.rds.amazonaws.com',
port = '5432',
database = 'yash_sa_schema')
#dataframe = psql.DataFrame("SELECT * FROM category", connection)
data = pd.read_sql_query('select * from yash_sa_schema.sa_file_ingst_log',con=connection)
print(data['published_file_path'].iloc[0])
#print(data['data_src_nm'].iloc[0])
print(data['published_file_name'].iloc[0])
print(data['Model'].iloc[0])
Type=data['file_upld_typ'].iloc[0]
model=data['Model'].iloc[0]
DatasetPath=data['published_file_path'].iloc[0]
filename=data['published_file_name'].iloc[0]
#DatasetPath=data['DatasetPath'].iloc[0]

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Using bucket sagemaker-us-east-1-754307369999


In [2]:
import pandas as pd
from sagemaker.sklearn.estimator import SKLearn
import time
import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.session import Session

sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()
import boto3
import pandas as pd
import time
if model=='AremaRetrain':
    client = boto3.client('s3')
    response = client.list_objects(
        Bucket='ptg-banking-poc',
        Delimiter='sa_published_zone/DS1/file_type=training/',
        Prefix='sa_published_zone/DS1/file_type=training/')
    #print(response)
    key=[]
    for obj in response['Contents']:
        keys=obj['Key'].replace('sa_published_zone/DS1/file_type=training/',"")
        key.append(keys)
        print(keys)
        filename=keys
        Type='Retraining'
        model='AremaRetrain'
        DatasetPath='s3://ptg-banking-poc/sa_published_zone/DS1/file_type=training/'+filename
        !aws s3 cp s3://sagemaker-us-east-1-754307369999/Models/AremaRetrain.py ./DataFolder/AremaRetrain.py
        time.sleep(2)
        sklearn_estimator = SKLearn(entry_point=model+'.py',
                                            train_instance_type='ml.m5.large',
                                            framework_version='0.20.0',
                                            source_dir='./DataFolder',
                                            role=role,
                                            hyperparameters={'filename': filename },
                                            enable_sagemaker_metrics=True)
        sklearn_estimator.fit({'train':DatasetPath})

                    # Deploy my estimator to a SageMaker Endpoint and get a Predictor
        predictor = sklearn_estimator.deploy(endpoint_name=model+'-endpoint-ML',instance_type='ml.m5.large',initial_instance_count=1)
elif model=='ChurnRetrain':
    import sys
    !{sys.executable} -m pip install sagemaker-experiments
    from sagemaker.sklearn.estimator import SKLearn
    import time

    import boto3
    import numpy as np
    import pandas as pd
    import boto3
    import json
    import sagemaker
    from sagemaker.session import Session
    from io import StringIO


    import sagemaker
    from sagemaker.session import Session
    from sagemaker.analytics import ExperimentAnalytics

    from smexperiments.experiment import Experiment
    from smexperiments.trial import Trial
    from smexperiments.trial_component import TrialComponent
    from smexperiments.tracker import Tracker
    from sagemaker.tensorflow import TensorFlow
    from sagemaker.sklearn.model import SKLearnModel


    sess = boto3.Session()
    sm = sess.client('sagemaker')
    role = sagemaker.get_execution_role()
    import argparse
    import os
    import sagemaker
    import boto3
    from sagemaker import get_execution_role
    region = boto3.Session().region_name

    session = sagemaker.Session()
    bucket = 'sagemaker-us-east-1-754307369999'
    prefix = 'sagemaker'
    role = get_execution_role()
    sm = boto3.Session().client(service_name='sagemaker',region_name=region)
    parameterName='batch_size+epochs'
    hidden_channel_trial_name_map = {}
    test = Experiment.create(experiment_name=f"sagemaker-Experiment-Trial-hidden-channels-{int(time.time())}",sagemaker_boto_client=sm)
    print(test)
    import itertools
    batch_size=[10,50]
    epochs=[20,50]
    parameterName='batch_size+epochs'
    l = list(itertools.product(batch_size,epochs))
    for batchsize, epochs in l:
        print(batchsize, epochs)        # create trial
        trial_name = f"cnn-training-job-{'Test'}-hidden-channels-{int(time.time())}"
        cnn_trial = Trial.create(
            trial_name=trial_name, 
            experiment_name=test.experiment_name,
            sagemaker_boto_client=sm,
        )
        hidden_channel_trial_name_map[parameterName] = trial_name

            # associate the proprocessing trial component with the current tria    
            # all input configurations, parameters, and metrics specified in estimator 
            # definition are automatically tracked

        model_name='churnRetrain'
        script_path = model_name+'.py'

        sagemaker_session = sagemaker.Session()
        tf_estimator = TensorFlow(
            entry_point=script_path,
            source_dir='./DataFolder',
            train_instance_count=1, 
            framework_version='1.12', 
            py_version='py3',
            base_job_name=model_name+f"-parameterName-hidden-channels-{int(time.time())}",
            train_instance_type="ml.m5.large",
            role=role,
            sagemaker_session=sagemaker_session,
            hyperparameters={'batchsize': str(batchsize),'epochs': str(epochs),'filename': filename },
            enable_sagemaker_metrics=True)
        tf_estimator.fit(
            inputs={'train':DatasetPath }, 
            experiment_config={
                "TrialName": cnn_trial.trial_name,
                "TrialComponentDisplayName": "Training",
            },
            wait=True,
        )
        print(tf_estimator)
            # Now associate the estimator with the Experiment and Trial
            # give it a while before dispatching the next training job
        time.sleep(2)



    client = boto3.client('s3')

    response = client.list_objects(
        Bucket='sagemaker-us-east-1-754307369999',
        Delimiter='Accuracy-DecisionTree/ChurnAccuracy/',
        Prefix='Accuracy-DecisionTree/ChurnAccuracy/')
    #print(response)
    key=[]
    for obj in response['Contents']:
        keys=obj['Key'].replace('sagemaker/',"")
        key.append(keys)
    li=[]
    print(key)
    key.pop(0) 
    for filename in key:
        df = pd.read_csv('s3://sagemaker-us-east-1-754307369999/'+filename, index_col=None, header=0)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    print(frame)
    df[df['MAPE']==df['MAPE'].max()]
    print(df['Jobname'].iloc[0])
    BestRun=df['Jobname'].iloc[0]
    sklearn_model = SKLearnModel(model_data="s3://sagemaker-us-east-1-754307369999/"+BestRun+"/output/model.tar.gz",source_dir='./DataFolder', role=role,
        entry_point="churnRetrain.py")

    predictor = sklearn_model.deploy(endpoint_name='Churnretrain-endpoint-ML',instance_type="ml.m5.large", initial_instance_count=1)
elif model=="LinearRetrain":
    import sys
    !{sys.executable} -m pip install sagemaker-experiments
    from sagemaker.sklearn.estimator import SKLearn
    import time

    import boto3
    import numpy as np
    import pandas as pd


    import sagemaker
    from sagemaker.session import Session
    from sagemaker.analytics import ExperimentAnalytics

    from smexperiments.experiment import Experiment
    from smexperiments.trial import Trial
    from smexperiments.trial_component import TrialComponent
    from smexperiments.tracker import Tracker

    sess = boto3.Session()
    sm = sess.client('sagemaker')
    role = sagemaker.get_execution_role()
    import argparse
    import os
    import sagemaker
    import boto3
    from sagemaker import get_execution_role
    region = boto3.Session().region_name

    session = sagemaker.Session()
    bucket = 'sagemaker-us-east-1-754307369999'
    prefix = 'sagemaker'
    role = get_execution_role()
    sm = boto3.Session().client(service_name='sagemaker',region_name=region)
    parameterName='batch_size+epochs'
    hidden_channel_trial_name_map = {}
    test = Experiment.create(experiment_name=f"sagemaker-Experiment-Trial-hidden-channels-{int(time.time())}",sagemaker_boto_client=sm)
    print(test)
    import itertools
    #estimators=[100,200,1000]
    min_samples_leaf=[10, 300, 5000]
    max_features = ["auto","sqrt"]
    parameterName='max_features'
    l = list(itertools.product(max_features,min_samples_leaf))
    for max_features,min_samples_leaf in l:
        print(max_features,min_samples_leaf) 
        #print(estimators)        # create trial
        trial_name = f"cnn-training-job-{'Test'}-hidden-channels-{int(time.time())}"
        cnn_trial = Trial.create(
            trial_name=trial_name, 
            experiment_name=test.experiment_name,
            sagemaker_boto_client=sm,
        )
        hidden_channel_trial_name_map[parameterName] = trial_name

            # associate the proprocessing trial component with the current tria    
            # all input configurations, parameters, and metrics specified in estimator 
            # definition are automatically tracked

        model_name='lrNewmodel'
        script_path = model_name+'.py'

        sagemaker_session = sagemaker.Session()
        sklearn = SKLearn(
            entry_point=script_path,
            source_dir='./DataFolder',
            base_job_name=model_name+f"-parameterName-hidden-channels-{int(time.time())}",
            train_instance_type="ml.m5.large",
            role=role,
            sagemaker_session=sagemaker_session,
            hyperparameters={'maxfeatures':max_features,'sampleleaf':min_samples_leaf,'filename': filename},
            enable_sagemaker_metrics=True)
        sklearn.fit(
            inputs={'train':DatasetPath }, 
            experiment_config={
                "TrialName": cnn_trial.trial_name,
                "TrialComponentDisplayName": "Training",
            },
            wait=True,
        )
        print(sklearn)
            # Now associate the estimator with the Experiment and Trial
            # give it a while before dispatching the next training job
        time.sleep(2)
    
    import boto3
    import pandas as pd
    from sagemaker.sklearn.model import SKLearnModel


    client = boto3.client('s3')

    response = client.list_objects(
        Bucket='sagemaker-us-east-1-754307369999',
        Delimiter='Accuracy-DecisionTree/LinearRegresion/',
        Prefix='Accuracy-DecisionTree/LinearRegresion/')
    #print(response)
    key=[]
    for obj in response['Contents']:
        keys=obj['Key'].replace('sagemaker/',"")
        key.append(keys)
    li=[]
    print(key)
    key.pop(0) 
    for filename in key:
        df = pd.read_csv('s3://sagemaker-us-east-1-754307369999/'+filename, index_col=None, header=0)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    print(frame)
    df[df['MAPE']==df['MAPE'].min()]
    df
    sm = boto3.Session().client(service_name='sagemaker',region_name='us-east-1')
    BestRun=frame['Jobname'].iloc[0]
    print(BestRun)
    sklearn_model = SKLearnModel(model_data="s3://sagemaker-us-east-1-754307369999/"+BestRun+"/output/model.tar.gz",source_dir='./DataFolder', role=role,
        entry_point="lrNewmodel.py")

    predictor = sklearn_model.deploy(endpoint_name='LinearRetrain-endpoint-ML',instance_type="ml.m5.large", initial_instance_count=1)

        




part-00000-0df7fcaf-2592-442e-b788-a117e9a1554f.c000.csv
download: s3://sagemaker-us-east-1-754307369999/Models/AremaRetrain.py to DataFolder/AremaRetrain.py
2020-04-05 13:32:53 Starting - Starting the training job...
2020-04-05 13:32:55 Starting - Launching requested ML instances......
2020-04-05 13:33:55 Starting - Preparing the instances for training...
2020-04-05 13:34:38 Downloading - Downloading input data...
2020-04-05 13:35:02 Training - Downloading the training image.....
2020-04-05 13:36:03 Training - Training image download completed. Training in progress.2020-04-05 13:36:03,720 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-04-05 13:36:03,723 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-05 13:36:03,733 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-04-05 13:36:03,998 sagemaker-containers INFO     Module AremaRetrain does not provide a setup.py. 
Generating 

2020-04-05 13:36:56,488 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-05 13:36:56,498 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_sklearn_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "filename": "part-00000-0df7fcaf-2592-442e-b788-a117e9a1554f.c000.csv"
    },
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {
        "train": {
            "TrainingInputMode": "File",
            "S3DistributionType": "FullyReplicated",
            "RecordWrapperType": "None"
        }
    },
    "input_dir": "/opt/ml/input",
    "is_master": true,
    "job_name": "sagemaker-scikit-learn-2020-04-05-13-32-53-063",
    "log_level": 20,
    "master_hostname": "algo-1",
    "model_di


2020-04-05 13:37:18 Uploading - Uploading generated training model
2020-04-05 13:37:18 Completed - Training job completed
Training seconds: 160
Billable seconds: 160
-------------!part-00000-ee64c38d-eeba-4d62-88f1-56a99920baa9.c000.csv
download: s3://sagemaker-us-east-1-754307369999/Models/AremaRetrain.py to DataFolder/AremaRetrain.py
2020-04-05 13:44:11 Starting - Starting the training job...
2020-04-05 13:44:13 Starting - Launching requested ML instances.........
2020-04-05 13:45:44 Starting - Preparing the instances for training...
2020-04-05 13:46:27 Downloading - Downloading input data...
2020-04-05 13:47:00 Training - Downloading the training image.2020-04-05 13:47:14,803 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-04-05 13:47:14,805 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-05 13:47:14,815 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-04-05 13:47:15,100

  Found existing installation: scikit-learn 0.20.0
    Uninstalling scikit-learn-0.20.0:
      Successfully uninstalled scikit-learn-0.20.0
2020-04-05 13:48:06,193 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-05 13:48:06,204 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_sklearn_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "filename": "part-00000-ee64c38d-eeba-4d62-88f1-56a99920baa9.c000.csv"
    },
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {
        "train": {
            "TrainingInputMode": "File",
            "S3DistributionType": "FullyReplicated",
            "RecordWrapperType": "None"
        }
    },
    "input_dir": "/opt/ml/input",
    "is_master":

KeyboardInterrupt: 

In [3]:
import boto3
client = boto3.client('sagemaker')
response = client.delete_endpoint(
    EndpointName=model+'-endpoint--Banking-SA'
)
print(response)

{'ResponseMetadata': {'RequestId': '8398c2f7-88f1-4828-b0f3-6d0c47d3a666', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8398c2f7-88f1-4828-b0f3-6d0c47d3a666', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Mon, 06 Apr 2020 12:08:33 GMT'}, 'RetryAttempts': 0}}
